In [159]:
import requests
import urllib
import datetime
from datetime import timedelta
import io
from bs4 import BeautifulSoup
import pandas as pd

In [3]:
req = requests.get("https://simple.wikipedia.org/wiki/List_of_European_countries")
soup = BeautifulSoup(req.content, 'html.parser')
european_countries = []
for ele in soup.findAll("td"):
    if "title=" in str(ele).strip() and ele.text.startswith(" "):
        if "[" in ele.text.strip():
            european_countries.append(ele.text[0:len(ele.text)-5].strip())
        else: 
            european_countries.append(ele.text.lstrip().replace("\n",""))

In [4]:
from datetime import date
today = date.today()
csvname = today.strftime('%m-%d-%Y')+".csv"
url = "https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_daily_reports/"+csvname

In [5]:
req = requests.get(url)
if(req.status_code == 404):
    today = today - timedelta(days = 1)
    csvname = today.strftime('%m-%d-%Y')+".csv"
    url = "https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_daily_reports/"+csvname
    req = requests.get(url)

In [6]:
req = requests.get(url)
if(req.status_code == 404):
    today = today - timedelta(days = 1)
    csvname = today.strftime('%m-%d-%Y')+".csv"
    url = "https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_daily_reports/"+csvname
    req = requests.get(url)

In [98]:
df = pd.read_csv(url)
df_cleaned = df[df["Province_State"].isnull()]
df_cleaned = df_cleaned[["Country_Region","Confirmed","Deaths","Lat","Long_"]]

In [99]:
europe = df_cleaned[df_cleaned["Country_Region"].isin(european_countries)]

In [212]:
europe.to_csv("data_europe.csv",index = False)

In [216]:
confirmed = pd.read_csv(
    "https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_global.csv")
death = pd.read_csv(
    "https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_global.csv")

In [246]:
name = today.strftime('X%m/X%d/%y').replace('X0', '')
confirmed = confirmed[confirmed["Province/State"].isnull()]
confirmed = confirmed[confirmed["Country/Region"].isin(european_countries)]
death = death[death["Province/State"].isnull()]
death = death[death["Country/Region"].isin(european_countries)]

'4/4/20'

In [218]:
confirmed.nlargest(5,confirmed.columns[-1]).to_csv("act_confirmed_europe.csv",index=False)
death.nlargest(5,death.columns[-1]).to_csv("act_death_europe.csv",index=False)

In [264]:
ratio=pd.merge(pd.DataFrame(confirmed[["Country/Region",name]]),
pd.DataFrame(death[["Country/Region",name]]),on="Country/Region")
ratio["ratio"]=(ratio.iloc[:,2]/ratio.iloc[:,1])*100
ratio.to_csv("ratio_europe.csv")